In [1]:
%load_ext autoreload
%autoreload 2
from src.data_pipelines.external.util import APIHttpClient, ClientsBaseURLs

In [2]:
ClientsBaseURLs.dwd

'https://opendata.dwd.de/'

In [3]:
getter = APIHttpClient(ClientsBaseURLs.dwd)

In [21]:
response = getter.get(endpoint="weather/local_forecasts/mos/MOSMIX_L/single_stations/01001/kml/MOSMIX_L_LATEST_01001.kmz")

In [62]:
type(response.content)

bytes

In [59]:
import io
import zipfile
from datetime import datetime, timezone
from typing import Dict, List, Optional, Any, Union
from xml.etree import ElementTree as ET

class DWDMosmixLSingleStationKMZParser:
    """Parser for DWD MOSMIX-L single station KMZ forecasts with raw data preservation."""
    
    # XML namespaces
    NS = {
        "kml": "http://www.opengis.net/kml/2.2",
        "dwd": "https://opendata.dwd.de/weather/lib/pointforecast_dwd_extension_V1_0.xsd"
    }
    
    def __init__(self, kmz_content: bytes):
        """
        Initialize parser with KMZ binary content.
        
        Args:
            kmz_content: Binary content of the KMZ file from HTTP response
        """
        self.kmz_content = kmz_content
        self.issue_time: Optional[str] = None  # Keep as raw string
        self.timestamps: List[str] = []        # Keep as raw strings
        self.forecasts: List[Dict[str, Any]] = []
        
        # Parse content on initialization
        self._parse()
    
    def _extract_kml(self) -> str:
        """Extract KML content from KMZ archive."""
        with io.BytesIO(self.kmz_content) as bio:
            with zipfile.ZipFile(bio) as kmz:
                if not kmz.filelist:
                    raise ValueError("Empty KMZ archive")
                with kmz.open(kmz.filelist[0]) as kml_file:
                    return kml_file.read().decode("ISO-8859-1")
    
    def _parse(self):
        """Main parsing method preserving raw data values."""
        kml_content = self._extract_kml()
        root = ET.fromstring(kml_content)
        
        # Parse issue time (keep as raw string)
        issue_time_elem = root.find(".//dwd:IssueTime", self.NS)
        if issue_time_elem is not None and issue_time_elem.text:
            self.issue_time = issue_time_elem.text.strip()
        
        # Parse forecast timestamps (keep as raw strings)
        timesteps_elem = root.find(".//dwd:ForecastTimeSteps", self.NS)
        if timesteps_elem is not None:
            for ts_elem in timesteps_elem.findall("dwd:TimeStep", self.NS):
                if ts_elem.text:
                    self.timestamps.append(ts_elem.text.strip())
        
        # Parse forecasts from the first Placemark
        placemark = root.find(".//kml:Placemark", self.NS)
        if placemark is not None:
            self._parse_forecasts(placemark)
    
    def _parse_forecasts(self, placemark: ET.Element):
        """Parse forecast data preserving raw values."""
        # Prepare empty forecasts structure
        forecast_data = [{"timestamp": ts} for ts in self.timestamps]
        
        # Process each forecast parameter
        for forecast_elem in placemark.findall(".//dwd:Forecast", self.NS):
            # Get parameter name
            param_name = forecast_elem.attrib.get(f"{{{self.NS['dwd']}}}elementName")
            if param_name not in ("RR1c", "RR3c"):
                continue
            
            # Get values string
            value_elem = forecast_elem.find("dwd:value", self.NS)
            if value_elem is None or value_elem.text is None:
                continue
                
            # Split values and keep as raw strings
            values = [v.strip() for v in value_elem.text.split()]
            
            # Assign values to timestamps
            for i in range(min(len(forecast_data), len(values))):
                forecast_data[i][param_name] = values[i]
        
        self.forecasts = forecast_data
    
    def get_issue_time(self) -> Optional[str]:
        """Get raw issue time string."""
        return self.issue_time
    
    def get_timestamps(self) -> List[str]:
        """Get raw timestamp strings."""
        return self.timestamps
    
    def get_forecasts(self) -> List[Dict[str, Any]]:
        """
        Get forecasts with raw values.
        
        Returns:
            List of forecast entries with timestamp and parameter values
        """
        return self.forecasts
    
    def to_json(self) -> Dict[str, Any]:
        """Convert parsed data to JSON-serializable dictionary with raw values."""
        return {
            "issue_time": self.issue_time,
            "forecasts": self.forecasts
        }

In [60]:
parser = DWDMosmixLSingleStationKMZParser(response.content)

# Parse content with raw values
parser = DWDMosmixLSingleStationKMZParser(response.content)

# Access raw data
issue_time = parser.get_issue_time()  # "2023-06-15T10:00:00.000Z"
timestamps = parser.get_timestamps()  # ["2023-06-15T12:00:00.000Z", ...]
forecasts = parser.get_forecasts()    # List of raw forecast dictionaries

# Get JSON representation
json_data = parser.to_json()  # Contains all raw values

In [61]:
json_data

{'issue_time': '2025-07-15T09:00:00.000Z',
 'forecasts': [{'timestamp': '2025-07-15T10:00:00.000Z',
   'RR1c': '0.00',
   'RR3c': '-'},
  {'timestamp': '2025-07-15T11:00:00.000Z', 'RR1c': '0.00', 'RR3c': '-'},
  {'timestamp': '2025-07-15T12:00:00.000Z', 'RR1c': '0.00', 'RR3c': '0.00'},
  {'timestamp': '2025-07-15T13:00:00.000Z', 'RR1c': '0.00', 'RR3c': '-'},
  {'timestamp': '2025-07-15T14:00:00.000Z', 'RR1c': '0.00', 'RR3c': '-'},
  {'timestamp': '2025-07-15T15:00:00.000Z', 'RR1c': '0.00', 'RR3c': '0.00'},
  {'timestamp': '2025-07-15T16:00:00.000Z', 'RR1c': '0.00', 'RR3c': '-'},
  {'timestamp': '2025-07-15T17:00:00.000Z', 'RR1c': '0.00', 'RR3c': '-'},
  {'timestamp': '2025-07-15T18:00:00.000Z', 'RR1c': '0.00', 'RR3c': '0.00'},
  {'timestamp': '2025-07-15T19:00:00.000Z', 'RR1c': '-', 'RR3c': '-'},
  {'timestamp': '2025-07-15T20:00:00.000Z', 'RR1c': '-', 'RR3c': '-'},
  {'timestamp': '2025-07-15T21:00:00.000Z', 'RR1c': '-', 'RR3c': '0.00'},
  {'timestamp': '2025-07-15T22:00:00.000Z', 'RR

In [42]:
#!/usr/bin/env python3
"""
DWD MOSMIX Parser for Jupyter Notebook
Adapted from the original command-line version for interactive use
"""

import json
import sys
from datetime import datetime, timezone
from locale import setlocale, LC_ALL
from pathlib import Path
from zipfile import ZipFile, BadZipFile
from contextlib import contextmanager
from typing import Optional, List, IO, Tuple, Dict, Generator, ClassVar, Set, Iterator, Any, Iterable, Union, Literal

try:
    from lxml.etree import iterparse, _Element as Element
except ModuleNotFoundError:
    from xml.etree.ElementTree import iterparse, Element  # type: ignore[assignment]


class TimezoneFinder:
    def timezone_at(self, **kwargs) -> Optional[str]:
        return None

    @classmethod
    def get_inst(cls, enabled: bool) -> "TimezoneFinder":
        if not enabled:
            return cls()
        try:
            from timezonefinder import TimezoneFinder as TF  # import only when needed at runtime due to overhead
            return TF(in_memory=True)  # type: ignore
        except ModuleNotFoundError:
            return cls()


class DwdMosmixParser:
    """
    Parsing methods for DWD MOSMIX KML XML files, namely either:
      * list of timestamps from ``ForecastTimeSteps``
      * properties of stations in ``Placemark``
      * value series in ``Forecast``
    Note that all methods iteratively consume from an i/o stream, such that it cannot be reused without rewinding it.
    """

    _ns: ClassVar[Dict[str, str]] = {  # abbreviations for used XML namespaces for readability
        "kml": r"http://www.opengis.net/kml/2.2",
        "dwd": r"https://opendata.dwd.de/weather/lib/pointforecast_dwd_extension_V1_0.xsd",
    }
    _undef_sign: ClassVar[str] = "-"  # dwd:FormatCfg/dwd:DefaultUndefSign

    @classmethod
    def _iter_tag(cls, fp: IO[bytes], tag: str) -> Iterator[Element]:
        if ":" in tag:
            ns, tag = tag.split(":", maxsplit=1)
            tag = f"{{{cls._ns[ns]}}}{tag}"
        for _evt, elem in iterparse(fp, events=["end"]):  # type: str, Element
            if elem.tag == tag:
                yield elem
                elem.clear()

    @classmethod
    def _parse_timestamp(cls, value: Optional[str]) -> int:
        if not value:
            raise ValueError("Undefined timestamp")
        try:
            return int(datetime.strptime(value, "%Y-%m-%dT%H:%M:%S.000Z").replace(tzinfo=timezone.utc).timestamp())
        except ValueError as e:
            raise ValueError(f"Cannot parse timestamp '{value}'") from e

    def parse_timestamps(self, fp: IO[bytes]) -> Iterator[int]:
        """Give all ``ForecastTimeSteps`` as integer timestamps."""
        for elem in self._iter_tag(fp, "dwd:ForecastTimeSteps"):
            yield from (self._parse_timestamp(_.text) for _ in elem.iterfind("dwd:TimeStep", self._ns))
            break

    @classmethod
    def _parse_coordinates(cls, value: str) -> Tuple[float, float, float]:
        values: List[str] = value.split(",")
        if len(values) != 3:
            raise ValueError(f"Cannot parse coordinates '{value}'")
        try:
            return float(values[0]), float(values[1]), float(values[2])
        except ValueError as e:
            raise ValueError(f"Cannot parse coordinates '{value}'") from e

    @classmethod
    def _parse_description(cls, placemark: Element) -> str:
        description: Optional[Element] = placemark.find("kml:description", cls._ns)
        if description is None or not description.text:
            raise ValueError("No 'Placemark.description' found")
        return description.text

    @classmethod
    def _parse_placemark(cls, placemark: Element) -> Dict[str, Any]:
        name: Optional[Element] = placemark.find("kml:name", cls._ns)
        if name is None or not name.text:
            raise ValueError("No 'Placemark.name' found")

        coordinates: Optional[Element] = placemark.find("kml:Point/kml:coordinates", cls._ns)
        if coordinates is None or not coordinates.text:
            raise ValueError("No 'Placemark.Point.coordinates' found")
        lng, lat, ele = cls._parse_coordinates(coordinates.text)

        return {
            "desc": cls._parse_description(placemark),
            "name": name.text,
            "lat": lat,
            "lng": lng,
            "ele": ele,
        }

    def parse_placemarks(self, fp: IO[bytes], timezones: bool) -> Iterator[Dict[str, Any]]:
        """Give all stations with their properties from ``Placemark`` nodes."""
        tf: TimezoneFinder = TimezoneFinder.get_inst(timezones)
        for elem in self._iter_tag(fp, "kml:Placemark"):
            placemark: Dict[str, Any] = self._parse_placemark(elem)
            placemark["tz"] = tf.timezone_at(lng=placemark["lng"], lat=placemark["lat"])
            yield placemark

    @classmethod
    def _parse_values(cls, values: str) -> List[Optional[float]]:
        try:
            return [float(_) if _ != cls._undef_sign else None for _ in values.split()]
        except ValueError as e:
            raise ValueError(f"Cannot parse forecast values '{values}'") from e

    @classmethod
    def _parse_forecast(cls, placemark: Element) -> Dict[str, List[Optional[float]]]:
        forecasts: Dict[str, List[Optional[float]]] = {}
        for forecast in placemark.iterfind("kml:ExtendedData/dwd:Forecast", cls._ns):
            name: Optional[Union[str, bytes]] = forecast.attrib.get(f"{{{cls._ns['dwd']}}}elementName")
            if not isinstance(name, str) or not name:
                raise ValueError("No 'Forecast.elementName' found")

            value: Optional[Element] = forecast.find("dwd:value", cls._ns)
            if value is None or not value.text:
                raise ValueError("No 'Forecast.value' found")

            forecasts[name] = cls._parse_values(value.text)
        return forecasts

    def parse_forecasts(self, fp: IO[bytes],
                        stations: Optional[Set[str]]) -> Iterator[Tuple[str, Dict[str, List[Optional[float]]]]]:
        """Give all value series in ``Forecast``, optionally limited to certain stations."""
        for elem in self._iter_tag(fp, "kml:Placemark"):
            station: str = self._parse_description(elem)
            if stations is None or station in stations:
                yield station, self._parse_forecast(elem)


@contextmanager
def kmz_reader(fp: IO[bytes]) -> Generator[IO[bytes], None, None]:
    """
    Wrap reading from *.kmz files, which are merely compressed *.kml (XML) files.
    """
    try:
        with ZipFile(fp) as zf:
            if len(zf.filelist) != 1:
                raise OSError(f"Unexpected archive contents: {' '.join(zf.namelist())}")
            with zf.open(zf.filelist[0]) as zp:
                yield zp
    except BadZipFile as e:
        raise OSError(str(e)) from None


@contextmanager
def kml_reader(filename: Path, compressed: Optional[bool] = None) -> Generator[IO[bytes], None, None]:
    """
    Read access for *.kml or compressed *.kmz files.
    """
    with open(filename, "rb") as fp:
        if compressed is True or (compressed is None and filename.suffix == ".kmz"):
            with kmz_reader(fp) as zp:
                yield zp
        else:
            yield fp


# Jupyter-friendly wrapper functions
class DwdMosmixNotebook:
    """
    Jupyter notebook-friendly wrapper for DWD MOSMIX parser
    """
    
    def __init__(self, file_path: str):
        """
        Initialize with path to KMZ/KML file
        
        Args:
            file_path: Path to the MOSMIX KMZ or KML file
        """
        self.file_path = Path(file_path)
        self.parser = DwdMosmixParser()
        setlocale(LC_ALL, "C")  # for strptime
        
    def get_timestamps(self) -> List[int]:
        """
        Parse and return all forecast timestamps as Unix timestamps
        
        Returns:
            List of Unix timestamps
        """
        with kml_reader(self.file_path) as fp:
            return list(self.parser.parse_timestamps(fp))
    
    def get_timestamps_readable(self) -> List[str]:
        """
        Parse and return all forecast timestamps in human-readable format
        
        Returns:
            List of timestamp strings in ISO format
        """
        timestamps = self.get_timestamps()
        return [datetime.fromtimestamp(ts, tz=timezone.utc).isoformat() for ts in timestamps]
    
    def get_stations(self, include_timezones: bool = False) -> List[Dict[str, Any]]:
        """
        Parse and return all weather stations information
        
        Args:
            include_timezones: Whether to determine timezones from coordinates
            
        Returns:
            List of station dictionaries with keys: desc, name, lat, lng, ele, tz
        """
        with kml_reader(self.file_path) as fp:
            return list(self.parser.parse_placemarks(fp, include_timezones))
    
    def get_forecasts(self, station_ids: Optional[List[str]] = None) -> Dict[str, Dict[str, List[Optional[float]]]]:
        """
        Parse and return forecast data for all or specified stations
        
        Args:
            station_ids: Optional list of station IDs to limit results to
            
        Returns:
            Dictionary mapping station IDs to their forecast data
        """
        stations_set = set(station_ids) if station_ids else None
        with kml_reader(self.file_path) as fp:
            return dict(self.parser.parse_forecasts(fp, stations_set))
    
    def save_to_json(self, data: Any, output_path: str) -> None:
        """
        Save data to JSON file
        
        Args:
            data: Data to save
            output_path: Path to output JSON file
        """
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    
    def get_station_by_name(self, name: str, include_timezones: bool = False) -> Optional[Dict[str, Any]]:
        """
        Find a station by name
        
        Args:
            name: Station name to search for
            include_timezones: Whether to determine timezones from coordinates
            
        Returns:
            Station dictionary or None if not found
        """
        stations = self.get_stations(include_timezones)
        for station in stations:
            if name.lower() in station['name'].lower():
                return station
        return None
    
    def get_stations_near(self, lat: float, lng: float, max_distance_km: float = 50.0) -> List[Dict[str, Any]]:
        """
        Find stations within a certain distance of given coordinates
        
        Args:
            lat: Latitude
            lng: Longitude
            max_distance_km: Maximum distance in kilometers
            
        Returns:
            List of stations within the specified distance
        """
        import math
        
        def haversine_distance(lat1, lon1, lat2, lon2):
            """Calculate the great circle distance between two points on Earth"""
            R = 6371  # Earth's radius in km
            
            lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            
            a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
            c = 2 * math.asin(math.sqrt(a))
            
            return R * c
        
        stations = self.get_stations()
        nearby_stations = []
        
        for station in stations:
            distance = haversine_distance(lat, lng, station['lat'], station['lng'])
            if distance <= max_distance_km:
                station['distance_km'] = round(distance, 2)
                nearby_stations.append(station)
        
        # Sort by distance
        nearby_stations.sort(key=lambda x: x['distance_km'])
        return nearby_stations


# Example usage functions for the notebook
def example_usage():
    """
    Example usage of the DWD MOSMIX parser in Jupyter notebook
    """
    print("Example usage:")
    print("1. Initialize parser:")
    print("   parser = DwdMosmixNotebook('path/to/your/file.kmz')")
    print()
    print("2. Get timestamps:")
    print("   timestamps = parser.get_timestamps_readable()")
    print("   print(f'Found {len(timestamps)} forecast timestamps')")
    print()
    print("3. Get stations:")
    print("   stations = parser.get_stations()")
    print("   print(f'Found {len(stations)} weather stations')")
    print()
    print("4. Get forecasts for specific stations:")
    print("   forecasts = parser.get_forecasts(['station_id_1', 'station_id_2'])")
    print()
    print("5. Find nearby stations:")
    print("   nearby = parser.get_stations_near(52.52, 13.41, max_distance_km=100)")
    print()
    print("6. Save data to JSON:")
    print("   parser.save_to_json(stations, 'stations.json')")

if __name__ == "__main__":
    example_usage()

Example usage:
1. Initialize parser:
   parser = DwdMosmixNotebook('path/to/your/file.kmz')

2. Get timestamps:
   timestamps = parser.get_timestamps_readable()
   print(f'Found {len(timestamps)} forecast timestamps')

3. Get stations:
   stations = parser.get_stations()
   print(f'Found {len(stations)} weather stations')

4. Get forecasts for specific stations:
   forecasts = parser.get_forecasts(['station_id_1', 'station_id_2'])

5. Find nearby stations:
   nearby = parser.get_stations_near(52.52, 13.41, max_distance_km=100)

6. Save data to JSON:
   parser.save_to_json(stations, 'stations.json')


In [43]:
# DWD MOSMIX Parser - Example Jupyter Notebook

# First, install required dependencies if needed:
# !pip install lxml
# !pip install timezonefinder  # Optional, for timezone support

# Import the necessary modules
import json
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

# Copy the DwdMosmixNotebook class code from the previous cell here
# ... (include the full parser code from the previous artifact)

# Example 1: Basic usage
print("=== DWD MOSMIX Parser Example ===")

# Initialize the parser with your KMZ/KML file
# Replace 'your_file.kmz' with the actual path to your DWD MOSMIX file
file_path = "MOSMIX_L_LATEST_01001(1).kmz"  # or "your_file.kml"

try:
    parser = DwdMosmixNotebook(file_path)
    print(f"✓ Successfully initialized parser with file: {file_path}")
except Exception as e:
    print(f"✗ Error initializing parser: {e}")
    print("Make sure your file path is correct and the file exists")

# Example 2: Get forecast timestamps
print("\n=== Getting Forecast Timestamps ===")
try:
    timestamps = parser.get_timestamps_readable()
    print(f"Found {len(timestamps)} forecast timestamps")
    print("First 5 timestamps:")
    for i, ts in enumerate(timestamps[:5]):
        print(f"  {i+1}. {ts}")
    if len(timestamps) > 5:
        print(f"  ... and {len(timestamps) - 5} more")
except Exception as e:
    print(f"Error getting timestamps: {e}")

# Example 3: Get weather stations
print("\n=== Getting Weather Stations ===")
try:
    stations = parser.get_stations(include_timezones=True)
    print(f"Found {len(stations)} weather stations")
    
    # Display first few stations
    print("\nFirst 3 stations:")
    for i, station in enumerate(stations[:3]):
        print(f"  {i+1}. {station['name']} ({station['desc']})")
        print(f"     Location: {station['lat']:.4f}°N, {station['lng']:.4f}°E")
        print(f"     Elevation: {station['ele']:.0f}m")
        if station['tz']:
            print(f"     Timezone: {station['tz']}")
        print()
        
except Exception as e:
    print(f"Error getting stations: {e}")

# Example 4: Convert stations to DataFrame for easier analysis
print("\n=== Converting Stations to DataFrame ===")
try:
    stations_df = pd.DataFrame(stations)
    print(f"DataFrame shape: {stations_df.shape}")
    print("\nColumn names:", stations_df.columns.tolist())
    print("\nFirst few rows:")
    print(stations_df.head())
    
    # Some basic statistics
    print(f"\nLatitude range: {stations_df['lat'].min():.2f}° to {stations_df['lat'].max():.2f}°")
    print(f"Longitude range: {stations_df['lng'].min():.2f}° to {stations_df['lng'].max():.2f}°")
    print(f"Elevation range: {stations_df['ele'].min():.0f}m to {stations_df['ele'].max():.0f}m")
    
except Exception as e:
    print(f"Error creating DataFrame: {e}")

# Example 5: Find stations by name
print("\n=== Finding Stations by Name ===")
try:
    # Search for stations containing "Berlin"
    berlin_station = parser.get_station_by_name("Berlin")
    if berlin_station:
        print(f"Found Berlin station: {berlin_station['name']}")
        print(f"  Location: {berlin_station['lat']:.4f}°N, {berlin_station['lng']:.4f}°E")
        print(f"  Station ID: {berlin_station['desc']}")
    else:
        print("No Berlin station found")
        
    # Search for stations containing "Munich" or "München"
    munich_station = parser.get_station_by_name("München")
    if munich_station:
        print(f"Found Munich station: {munich_station['name']}")
        print(f"  Location: {munich_station['lat']:.4f}°N, {munich_station['lng']:.4f}°E")
        print(f"  Station ID: {munich_station['desc']}")
    else:
        print("No Munich station found")
        
except Exception as e:
    print(f"Error finding stations: {e}")

# Example 6: Find nearby stations
print("\n=== Finding Nearby Stations ===")
try:
    # Find stations near Berlin (52.52°N, 13.41°E)
    nearby_stations = parser.get_stations_near(52.52, 13.41, max_distance_km=100)
    print(f"Found {len(nearby_stations)} stations within 100km of Berlin")
    
    print("\nClosest 5 stations to Berlin:")
    for i, station in enumerate(nearby_stations[:5]):
        print(f"  {i+1}. {station['name']} - {station['distance_km']}km away")
        print(f"     Location: {station['lat']:.4f}°N, {station['lng']:.4f}°E")
        print(f"     Station ID: {station['desc']}")
        print()
        
except Exception as e:
    print(f"Error finding nearby stations: {e}")

# Example 7: Get forecast data for specific stations
print("\n=== Getting Forecast Data ===")
try:
    # Get forecasts for the first 3 stations
    station_ids = [station['desc'] for station in stations[:3]]
    forecasts = parser.get_forecasts(station_ids)
    
    print(f"Retrieved forecasts for {len(forecasts)} stations")
    
    # Display forecast parameters for the first station
    first_station_id = list(forecasts.keys())[0]
    first_station_forecast = forecasts[first_station_id]
    
    print(f"\nForecast parameters for station {first_station_id}:")
    for param, values in first_station_forecast.items():
        non_null_values = [v for v in values if v is not None]
        print(f"  {param}: {len(values)} values ({len(non_null_values)} non-null)")
        if non_null_values:
            print(f"    Range: {min(non_null_values):.2f} to {max(non_null_values):.2f}")
    
    # Example: Create a simple forecast DataFrame for one station
    print(f"\n=== Creating Forecast DataFrame for {first_station_id} ===")
    
    # Combine timestamps with forecast data
    forecast_data = []
    for i, timestamp in enumerate(timestamps):
        row = {'timestamp': timestamp}
        for param, values in first_station_forecast.items():
            if i < len(values):
                row[param] = values[i]
        forecast_data.append(row)
    
    forecast_df = pd.DataFrame(forecast_data)
    print(f"Created DataFrame with {len(forecast_df)} rows and {len(forecast_df.columns)} columns")
    print("\nFirst few rows:")
    print(forecast_df.head())
    
except Exception as e:
    print(f"Error getting forecasts: {e}")

# Example 8: Save data to files
print("\n=== Saving Data to Files ===")
try:
    # Save stations to JSON
    parser.save_to_json(stations, 'stations.json')
    print("✓ Saved stations to 'stations.json'")
    
    # Save timestamps to JSON
    parser.save_to_json(timestamps, 'timestamps.json')
    print("✓ Saved timestamps to 'timestamps.json'")
    
    # Save forecasts to JSON
    # Note: This might be a large file depending on how many stations you have
    parser.save_to_json(forecasts, 'forecasts.json')
    print("✓ Saved forecasts to 'forecasts.json'")
    
    # Save stations DataFrame to CSV
    stations_df.to_csv('stations.csv', index=False)
    print("✓ Saved stations DataFrame to 'stations.csv'")
    
    # Save forecast DataFrame to CSV
    forecast_df.to_csv('forecast_sample.csv', index=False)
    print("✓ Saved sample forecast DataFrame to 'forecast_sample.csv'")
    
except Exception as e:
    print(f"Error saving files: {e}")

print("\n=== Done! ===")
print("Check the generated files in your current directory.")

=== DWD MOSMIX Parser Example ===
✓ Successfully initialized parser with file: MOSMIX_L_LATEST_01001(1).kmz

=== Getting Forecast Timestamps ===
Found 247 forecast timestamps
First 5 timestamps:
  1. 2025-07-15T04:00:00+00:00
  2. 2025-07-15T05:00:00+00:00
  3. 2025-07-15T06:00:00+00:00
  4. 2025-07-15T07:00:00+00:00
  5. 2025-07-15T08:00:00+00:00
  ... and 242 more

=== Getting Weather Stations ===
Found 1 weather stations

First 3 stations:
  1. 01001 (JAN MAYEN)
     Location: 70.9300°N, -8.6700°E
     Elevation: 10m
     Timezone: Europe/Oslo


=== Converting Stations to DataFrame ===
DataFrame shape: (1, 6)

Column names: ['desc', 'name', 'lat', 'lng', 'ele', 'tz']

First few rows:
        desc   name    lat   lng   ele           tz
0  JAN MAYEN  01001  70.93 -8.67  10.0  Europe/Oslo

Latitude range: 70.93° to 70.93°
Longitude range: -8.67° to -8.67°
Elevation range: 10m to 10m

=== Finding Stations by Name ===
No Berlin station found
No Munich station found

=== Finding Nearby St